## Is It a Cat or a Dog? (Image Classification)

Dataset Source: https://www.kaggle.com/datasets/erkamk/cat-and-dog-images-dataset

##### Import Necessary Libraries

In [0]:
import pyspark
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import DoubleType, ArrayType, StringType
import pyspark.sql.functions as F
from pyspark.ml import Pipeline

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

##### Versions of Libraries, Modules, Frameworks Used in This Project

In [0]:
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

Spark NLP version 4.2.6
Apache Spark version: 3.3.0


#### Create Functions Used Throughout Project

##### Function to Ingest Dataset

In [0]:
def ingest_folder_of_images_w_label(image_folders: [str], \
                                   labels: [str]):
    '''
    This function thats in a list of folder names that contain 
    images in it (not a list of folders with images in them) 
    and returns those images into a DataFrame.
    '''
    from pyspark.sql.dataframe import DataFrame
    
    for x in image_folders:
        temp_df: DataFrame = spark.read \
                .format("image") \
                .option("dropInvalid", value = True) \
                .load(x)
        
        temp_df = temp_df.withColumn("label", F.lit(labels[image_folders.index(x)]))
        
        if image_folders.index(x) > 0:
            df = df.union(temp_df)
        elif image_folders.index(x) == 0:
            df: DataFrame = temp_df
    return df

##### Create Pipeline Function (Creates Stages, Pipeline, Fit, & Transform)

In [0]:
def image_clf_model(dataset: pyspark.sql.dataframe.DataFrame, \
                   pretrained: str = "none" \
                   ) -> pyspark.sql.dataframe.DataFrame:
    '''
    This function creates the pipeline stages, the pipeline, 
    and it fits and transforms the dataset
    '''
    
    from pyspark.ml import Pipeline
    
    import sparknlp
    from sparknlp.base import ImageAssembler
    from sparknlp.annotator import ViTForImageClassification
    
    # Image Assembler
    img_assembler = ImageAssembler() \
        .setInputCol("image") \
        .setOutputCol("image_assembler")
    
    # Image Classifier
    if pretrained == "none":
        img_clf = ViTForImageClassification \
            .pretrained() \
            .setInputCols("image_assembler") \
            .setOutputCol("class")
    elif pretrained != "none":
        img_clf = ViTForImageClassification \
            .pretrained(pretrained, "en") \
            .setInputCols("image_assembler") \
            .setOutputCol("class")
    
    image_clf_pipe = Pipeline().setStages([
        img_assembler,
        img_clf
    ])
    
    predictions = image_clf_pipe.fit(dataset).transform(dataset)
    
    return predictions

##### Function for Evaluating Metrics

In [0]:
def mc_evaluate_with_spark_metrics(dataset: pyspark.sql.dataframe.DataFrame, \
                                metrics: [str], \
                                model_name: str, \
                                label_col: str = "label", \
                                predictionCol: str = "prediction" \
                               ) -> None:
    '''
    Calculate & display metrics for a multiclass classification analysis.
    '''
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    
    print("+---------------------------------------------+")
    print("|  " + model_name.center(41) + "  |")
    print("+---------------------------------------------+")
    print("|   %s  |  %s   |" % ("Metric".rjust(20), "Value".ljust(14)))
    print("+---------------------------------------------+")
    for x in metrics:
        evaluator = MulticlassClassificationEvaluator(labelCol=label_col, \
                                                      predictionCol=predictionCol, \
                                                      metricName=x) 
        score = evaluator.evaluate(dataset)
        print("|   %s  |  %s   |" % (x.rjust(20), str(round(score, 6)).ljust(14)))
        print("+---------------------------------------------+")

#### Actual Project

##### Ingest Training Dataset

In [0]:
training_data_files = ["/FileStore/tables/dogs_and_cats/cats", 
              "/FileStore/tables/dogs_and_cats/dogs"]

label_values = ["cats", "dogs"]

df: DataFrame = ingest_folder_of_images_w_label(training_data_files, label_values)

df = df.persist()

display(df)

image,label
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/359.png, 502, 671, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/63.png, 634, 491, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/58.png, 505, 473, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/149.png, 603, 465, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/70.png, 563, 567, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/62.png, 462, 469, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/72.png, 474, 433, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/360.png, 460, 365, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/76.png, 407, 426, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats
"List(dbfs:/FileStore/tables/dogs_and_cats/cats/75.png, 618, 483, 3, 16, Binary image data placeholder. Access the image data field directly to view raw binary data., ...)",cats


Output can only be rendered in Databricks

##### Prepare & Train Model as Well as Generate Predictions

In [0]:
predictions = image_clf_model(df, "image_classifier_vit_autotrain_cifar10__base")

image_classifier_vit_autotrain_cifar10__base download started this may take some time.
Approximate size to download 307 MB
[OK!]


##### Prepare Predictions For Metrics Evaluation Function

In [0]:
preds = predictions.select(F.col("label").alias("label"), F.col("class.result").getItem(0).alias("prediction"))

preds = preds.persist()

label_converter = {"cats" : "0", "dogs" : "1"}

preds = preds.replace(label_converter, subset=["label"])

display(preds)

label,prediction
0,cat
0,cat
0,cat
0,cat
0,cat
0,cat
0,cat
0,cat
0,cat
0,cat


##### Prepare Predictions For Metrics Evaluation Function (Part 2)

In [0]:
converter = {'airplane' : "-1", 
             'automobile' : "1", 
             'bird' : "-1", 
             'cat' : "0", 
             'deer' : "-1", 
             'dog' : "1", 
             'frog' : "-1", 
             'horse' : "-1", 
             'ship' : "2", 
             'truck' : "1"}

preds = preds.replace(converter)

#Converter String data type to Double data type
preds = preds.withColumn("label", F.col("label").cast(DoubleType())) \
            .withColumn("prediction", F.col("prediction").cast(DoubleType()))

display(preds)

label,prediction
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0
0.0,0.0


##### Calculate & Display Metrics

In [0]:
metrics_to_eval = ["accuracy", "f1", "weightedPrecision", "weightedRecall"]

mc_evaluate_with_spark_metrics(preds, \
                                metrics_to_eval, \
                                "Is it a Cat or Dog?", \
                                label_col = "label", \
                                predictionCol = "prediction")

+---------------------------------------------+
|             Is it a Cat or Dog?             |
+---------------------------------------------+
|                 Metric  |  Value            |
+---------------------------------------------+
|               accuracy  |  0.995            |
+---------------------------------------------+
|                     f1  |  0.995            |
+---------------------------------------------+
|      weightedPrecision  |  0.99505          |
+---------------------------------------------+
|         weightedRecall  |  0.995            |
+---------------------------------------------+


##### End Spark Session

In [0]:
df = df.unpersist()
preds = preds.unpersist()

spark.stop()

### Notes & Other Takeaways From This Project
****
- Only one wrong prediction out of the 200 samples. I would say that is pretty good. Now, I need to find a larger dataset to test!
****